In [1]:
from src import StudyGen

In [2]:
study = StudyGen(path_configuration='config.yaml', path_master='master.yaml')

In [3]:
study.generate_all()

ValueError: Parameter fact_a_bc is not defined in the configuration